In [2]:
# Импортируем библиотеки

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error

In [0]:
def split_sequence(sequence, n_steps):
    n = len(sequence)
    X, y = list(), list()
    for i in range(n):
        end_ix = i + n_steps
        if end_ix > n-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [0]:
def plot_difference(true_sequence, predict_values, unit):
    print(f'Prediction of model with {unit} units:')
    df = pd.DataFrame({'real value': true_sequence, 'predicted value': np.round(predict_values, 3), 'difference': np.round(abs(true_sequence-predict_values), 1)})
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(df)

# **Data generation**

$a_n = a_{n-3} + a_{n-2} + 2*a_{n-1}$

In [3]:
def data(start=(0, 1, 1), num=10):
    x = [start[0], start[1], start[2]]
    for i in range(3, num+1):
        x.append(2*x[i-1] + x[i-2] + x[i-3])
    return np.array(x)

In [0]:
def generate_data(n_steps=4):
    
    num = 15
    batch_size = 1000
    i = 0
    
    X = np.empty((batch_size, n_steps))
    y = np.empty((batch_size, 1))

    while i < batch_size:

        start_val = np.random.randint(0, 10, (3, ))
    
        array = data(start=(start_val[0], start_val[1], start_val[2]), num=num)
        j = 0

        while j <= num - (n_steps + 1):
            if i >= batch_size: break
            X[i, :] = array[j:n_steps+j]
            y[i, :] = array[j+n_steps] 
            i += 1
            j += 1
        
    _, index = np.unique(X, axis=0, return_index=True)
    X, y = X[index], y[index]
    X = X.reshape((X.shape[0], X.shape[1], 1))
    return X, y

In [0]:
n_steps = 6
X_train, y_train = generate_data(n_steps)
print(f'X_train shape is {X_train.shape}')
print(f'y_train shape is {y_train.shape}')

X_train shape is (965, 6, 1)
y_train shape is (965, 1)


## **Model with one LSTM layer**

In [0]:
def predict(true_sequence, model):
    predict_values = true_sequence[:n_steps]
    k = n_steps
    length = true_sequence.size
    while k != length:
        X = predict_values[-n_steps::]
        X = X.reshape((1, n_steps, 1))
        f_x = model.predict(X, verbose=0) 
        predict_values = np.append(predict_values, f_x)
        k += 1
    return predict_values

In [0]:
sequence = data(start=(1, 3, 2), num=15)

In [0]:
units_number = list(range(1, 11)) + list(range(15, 65, 5))
losses = []
min_loss = None
min_unit = None

for unit in units_number: 
    model = Sequential()
    model.add(LSTM(unit, activation='softplus', input_shape=(n_steps, 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    model.fit(X_train, y_train, epochs=600, validation_split=0.2, verbose=0)
    
    predict_y = predict(sequence, model)
    mse = mean_squared_error(sequence, predict_y)

    if min_loss is None or mse < min_loss:
        min_unit = unit
        min_loss = mse
        best_model = model
    losses.append(mse)
    
    print(f'Units in LSTM layer: {unit}, MSE is: {np.round(mse, 4)}')
    if unit in (5, 6, 7, 8, 9, 40, 50):
        plot_difference(sequence, predict_y, unit)

Units in LSTM layer: 1, MSE is: 26583466125.9725
Units in LSTM layer: 2, MSE is: 3167051.8795
Units in LSTM layer: 3, MSE is: 6366210.4932
Units in LSTM layer: 4, MSE is: 7303244.7909
Units in LSTM layer: 5, MSE is: 7967404.153
Prediction of model with 5 units:
    real value  predicted value  difference
0            1            1.000         0.0
1            3            3.000         0.0
2            2            2.000         0.0
3            8            8.000         0.0
4           21           21.000         0.0
5           52           52.000         0.0
6          133          126.830         6.2
7          339          328.512        10.5
8          863          859.055         3.9
9         2198         2160.840        37.2
10        5598         5488.157       109.8
11       14257        13889.817       367.2
12       36310        35537.988       772.0
13       92475        90726.508      1748.5
14      235517       231305.422      4211.6
15      599819       589526.312   

# **Testing best model performance**

Пусть $a_0=2, a_1=3, a_2=3 $

In [0]:
sequence = data(start=(2, 3, 3), num=15)
predict_y = predict(sequence, best_model)
mse = mean_squared_error(sequence, predict_y)
print(f'mse is {mse}')
plot_difference(sequence, predict_y, min_unit)

mse is 10195.098236567865
Prediction of model with 50 units:
    real value  predicted value  difference
0            2            2.000         0.0
1            3            3.000         0.0
2            3            3.000         0.0
3           11           11.000         0.0
4           28           28.000         0.0
5           70           70.000         0.0
6          179          179.329         0.3
7          456          455.122         0.9
8         1161         1160.398         0.6
9         2957         2954.723         2.3
10        7531         7521.495         9.5
11       19180        19188.176         8.2
12       48848        48860.109        12.1
13      124407       124413.656         6.7
14      316842       316738.719       103.3
15      806939       806549.000       390.0


Пусть $a_0=1, a_1=1, a_2=1$

In [0]:
sequence = data(start=(1, 1, 1), num=20)
predict_y = predict(sequence, best_model)
mse = mean_squared_error(sequence, predict_y)
print(f'mse is {mse}')
plot_difference(sequence, predict_y, min_unit)

mse is 8670890.65043752
Prediction of model with 50 units:
    real value  predicted value  difference
0            1     1.000000e+00         0.0
1            1     1.000000e+00         0.0
2            1     1.000000e+00         0.0
3            4     4.000000e+00         0.0
4           10     1.000000e+01         0.0
5           25     2.500000e+01         0.0
6           64     6.409000e+01         0.1
7          163     1.638020e+02         0.8
8          415     4.151370e+02         0.1
9         1057     1.057270e+03         0.3
10        2692     2.690704e+03         1.3
11        6856     6.853827e+03         2.2
12       17461     1.749903e+04        38.0
13       44470     4.454496e+04        75.0
14      113257     1.133921e+05       135.1
15      288445     2.886774e+05       232.4
16      734617     7.351466e+05       529.6
17     1870936     1.872456e+06      1520.5
18     4764934     4.768582e+06      3647.5
19    12135421     1.214184e+07      6416.0
20    30906712   

In [0]:
sequence = data(start=(6, 2, 6), num=15)
predict_y = predict(sequence, best_model)
mse = mean_squared_error(sequence, predict_y)
print(f'mse is {mse}')
plot_difference(sequence, predict_y, min_unit)

mse is 125717.42158397782
Prediction of model with 50 units:
    real value  predicted value  difference
0            6            6.000         0.0
1            2            2.000         0.0
2            6            6.000         0.0
3           20           20.000         0.0
4           48           48.000         0.0
5          122          122.000         0.0
6          312          311.277         0.7
7          794          792.192         1.8
8         2022         2020.158         1.8
9         5150         5148.531         1.5
10       13116        13101.656        14.3
11       33404        33387.898        16.1
12       85074        85045.719        28.3
13      216668       216529.188       138.8
14      551814       551374.312       439.7
15     1405370      1404029.250      1340.8


# **Model with two LSTM layers**

In [0]:
units_number = list(range(1, 11)) + list(range(15, 65, 5))
losses = []
min_loss = None
min_unit = None

for unit in units_number: 
    model = Sequential()
    model.add(LSTM(unit, activation='softplus', return_sequences=True, input_shape=(n_steps, 1)))
    model.add(LSTM(unit, activation='softplus'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    model.fit(X_train, y_train, epochs=600, validation_split=0.2, verbose=0)
    
    predict_y = predict(sequence, model)
    mse = mean_squared_error(sequence, predict_y)

    if min_loss is None or mse < min_loss:
        min_unit = unit
        min_loss = mse
        best_model = model
    losses.append(mse)
    
    print(f'Units in LSTM layer: {unit}, MSE is: {np.round(mse, 4)}')
    if unit in (5, 6, 7, 8, 9, 40, 50):
        plot_difference(sequence, predict_y, unit)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Units in LSTM layer: 1, MSE is: 757258475.31
Units in LSTM layer: 2, MSE is: 3577867.6925
Units in LSTM layer: 3, MSE is: 5863140.3093
Units in LSTM layer: 4, MSE is: 226197.2031
Units in LSTM layer: 5, MSE is: 5622365.3714
Prediction of model with 5 units:
    real value  predicted value  difference
0            1            1.000         0.0
1            3            3.000         0.0
2            2            2.000         0.0
3            8            8.000         0.0
4           21           21.000         0.0
5           52           52.000         0.0
6          133          126.819         6.2
7          339          335.286         3.7
8          863          848.888        14.1
9         2198         2174.811        23.2
10        5598         5514.600        83.4
11       14257        14009.901       2